In [ ]:
import os
import re
import io
import sys
import glob
import enum
import json
import dask
import xlrd
import base64
import time
import imageio
import requests
import datetime
import psycopg2
import numpy as np
import pandas as pd
import skimage

import dask.diagnostics
import sqlalchemy as db
import sqlalchemy.orm
import sqlalchemy.ext.declarative
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 1

sys.path.append('../..')
%aimport opencell.imaging.managers
%aimport opencell.imaging.processors
%aimport opencell.file_utils

from opencell import constants, file_utils
from opencell.cli import database as db_cli
from opencell.cli import imaging as imaging_cli
from opencell.database import models
from opencell.database import operations
from opencell.database import utils as db_utils
from opencell.imaging import utils as im_utils
from opencell.imaging import images, managers, processors, viz

In [ ]:
url = db_utils.url_from_credentials('../../db-credentials-dev.json')
url = db_utils.url_from_credentials('../../db-credentials-cap.json')
url

In [ ]:
engine = db.create_engine(url)
session_factory = db.orm.sessionmaker(bind=engine)
Session = db.orm.scoped_session(session_factory)
session = Session()

# reflect the cell_line_metadata view
clm = db.Table("cell_line_metadata", models.Base.metadata, autoload=True, autoload_with=engine)
fov_rank = db.Table("fov_rank", models.Base.metadata, autoload=True, autoload_with=engine)

### Append FACS grade and sequencing results to Manu's annotations CSV

This is on 2020-02-26 for Hera.

In [ ]:
d_raw = pd.read_csv('/Users/keith.cheveralls/2020-02-21-17-05-12_cell-line-annotations_ML.csv')

In [ ]:
list(d_raw.loc[d_raw.no_gfp == 1].target_name)

In [ ]:
with open('/Users/keith.cheveralls/projects/opencell-vis/src/demo/data/facs_grades.json', 'r') as file:
    facs_grades = json.load(file)

In [ ]:
facs_grades

In [ ]:
for ind, row in d.iterrows():
    d.at[ind, 'facs_grade'] = facs_grades.get('%s-%s' % (row.plate_id, row.well_id))

In [ ]:
d['hdr_all'] = None
d['hdr_modified'] = None

for ind, row in d.iterrows():
    try:
        sequencing = (
            Session.query(models.SequencingDataset)
            .filter(models.SequencingDataset.cell_line_id == row.cell_line_id)
            .first()
        )
        d.at[ind, 'hdr_all'] = sequencing.scalars.get('hdr_all')
        d.at[ind, 'hdr_modified'] = sequencing.scalars.get('hdr_modified')
    except Exception as error:
        print(row.cell_line_id)

In [ ]:
d.to_csv('/Users/keith.cheveralls/projects/opencell/cache/2020-02-26_annotations-w-facs-seq.csv')

In [ ]:
# list of No-GFP lines for Hiro
d_hiro = d_raw[['plate_id', 'well_id', 'target_name', 'cell_line_id', 'no_gfp']]
d_hiro.to_csv('/Users/keith.cheveralls/projects/opencell/cache/2020-03-03_no-gfp-for-hiro.csv')